In [1]:
win_size = 3

In [2]:
import sys
sys.path.append('../')
from time import sleep
from models.attention_series import Grade_regressor
import spectral
from configs.training_cfg import device
import torch
import ast
import numpy as np
from PIL import Image
import pandas as pd
from spectral import imshow
from matplotlib import rcParams
import seaborn as sns
from matplotlib.pyplot import MultipleLocator
import matplotlib.pyplot as plt

spectral.settings.envi_support_nonlowercase_params = True

rcParams['figure.figsize'] = 11,19

pre_list = []
erro_list = []

In [3]:
# 测试样本绝对误差
for k in range(1):
    model = Grade_regressor().to(device).eval()
    model.load_state_dict(torch.load("..\\ckpt\\(TSI)SiO2_0.93_balanced_100000.pt"))
    # model.load_state_dict(torch.load("..\\ckpt\\(MLP)NIR_30T_raw_blanced_68000.pt"))
    result = []
    err = 0
    err_count = 0 
    pool = torch.nn.AvgPool2d(win_size,win_size)
    mask_rgb_values = [[255,242,0],[34,177,76],[255,0,88],[184,61,186]]

    spec_id = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62]    # 选择评估的成像光谱图片编号


    for id in spec_id:
        img = spectral.envi.open("E:\\d盘备份\\近红外部分\\spectral_data\\{}-Radiance From Raw Data-Reflectance from Radiance Data and Measured Reference Spectrum.bip.hdr".format(id))

        # 根据模型使用波段选择
        # img_data = torch.Tensor(img.asarray()/6000)[:,:,:-4]
        img_data = torch.Tensor(img.asarray()/6000)[:,:,:]
        mask = np.array(Image.open("E:\\d盘备份\\近红外部分\\spectral_data\\{}-Radiance From Raw Data-Reflectance from Radiance Data and Measured Reference Spectrum.bip.hdr_mask.png".format(id)))
        if mask.shape[2] == 4:
            mask = mask[:,:,:-1]
        gt = ast.literal_eval(img.metadata['gt_SiO2'])
        gt = torch.Tensor(gt)
        gts = gt.__len__()

        with torch.no_grad():
            img_data = pool(img_data.permute(2,0,1)).permute(1,2,0)
            row,col,_ = img_data.shape
            heat_map = []

            for i in range(row):
                heat_map.append(model(img_data[i].to(device)).squeeze(1).unsqueeze(0).to("cpu"))   # 只评估Tfe
                torch.cuda.empty_cache()

        heat_map = torch.cat(heat_map, dim=0)

        # 可视化
        # plt.figure()#防止在上一张图上重新绘制
        # hm = sns.heatmap(heat_map.numpy(), cmap="viridis", xticklabels=False, yticklabels=False)
        # hm.figure.savefig("heatmap/{}号预测结果可视化.jpg".format(id))

        predict_sum = torch.zeros(gts)
        pixel_count = torch.ones(gts)/1e5

        for r in range(row):
            for c in range(col):
                for i in range(gts):
                    if mask[r*win_size+win_size//2,c*win_size+win_size//2].tolist() == mask_rgb_values[i]:
                        predict_sum[i] += heat_map[r,c] 
                        pixel_count[i] += 1

        prediction = predict_sum / pixel_count * 100

        err_list = ((prediction-gt)).tolist()
        pred_list = (prediction).tolist()
        for i in range(err_list.__len__()):
            erro_list.append(round(err_list[i], 2))
            pre_list.append(round(pred_list[i], 2))


    # avg_err = err/(err_count)
    # print("{}k  avgmse:{}".format(k+1, avg_err))

In [4]:
erro_list

[10.28,
 -3.09,
 0.52,
 0.55,
 2.21,
 5.91,
 -4.0,
 -0.08,
 4.83,
 0.0,
 -3.35,
 0.62,
 -0.57,
 -13.55,
 -1.79,
 7.45,
 -6.04,
 4.24,
 -5.41,
 -4.65,
 -14.04,
 -5.49,
 -7.24,
 -4.58,
 1.96,
 4.86,
 -5.99,
 0.52,
 8.89,
 4.54,
 1.71,
 -2.48,
 -3.9,
 -1.38,
 -0.37,
 0.39,
 -2.29,
 -0.86,
 1.93,
 -2.86,
 -7.2,
 4.81,
 -4.71,
 9.23,
 12.8,
 0.03,
 -4.7,
 7.67,
 2.04,
 12.68,
 4.4,
 -0.5,
 8.52,
 4.73,
 6.94,
 8.28,
 -11.9,
 4.49,
 13.16,
 -21.21,
 1.95,
 3.39,
 -4.59,
 0.73,
 2.42,
 1.16,
 1.56,
 3.46,
 6.33,
 5.84,
 -1.91,
 9.46,
 -4.15,
 -3.6,
 1.74,
 4.36,
 -0.15,
 0.38,
 -1.19,
 -17.9,
 2.21,
 -1.92,
 6.81,
 -0.8,
 10.19,
 2.71,
 2.74,
 -4.74,
 -6.14,
 -9.21,
 -6.51,
 -0.56,
 2.57,
 11.46,
 12.27,
 3.85,
 -4.35,
 1.05,
 2.19,
 10.66,
 -4.32,
 9.53,
 6.29,
 -12.11,
 -2.82,
 -5.86,
 8.78,
 2.16,
 0.69,
 -4.95,
 8.95,
 -7.49,
 5.17,
 2.71,
 7.54,
 3.46,
 4.83,
 -21.7,
 6.3,
 -6.93,
 -7.44,
 -5.18,
 -10.47,
 10.84,
 -10.75,
 9.05,
 -8.78,
 -23.36,
 -4.03,
 9.65,
 6.58,
 5.63,
 3.06,
 2.22,

In [5]:
pre_list

[57.67,
 50.19,
 55.59,
 50.51,
 56.28,
 57.03,
 58.77,
 72.54,
 64.56,
 65.78,
 72.23,
 63.39,
 47.77,
 64.9,
 54.31,
 56.26,
 64.61,
 57.43,
 60.22,
 64.92,
 49.11,
 59.92,
 63.7,
 48.37,
 58.19,
 61.58,
 66.92,
 59.37,
 65.75,
 67.32,
 82.37,
 53.61,
 73.87,
 79.28,
 57.12,
 81.05,
 81.62,
 72.51,
 57.7,
 80.4,
 67.87,
 60.13,
 73.74,
 64.24,
 59.48,
 76.33,
 64.21,
 58.6,
 68.58,
 68.23,
 64.77,
 65.84,
 66.18,
 63.44,
 63.03,
 69.4,
 33.78,
 61.98,
 65.4,
 34.08,
 7.78,
 75.55,
 61.95,
 6.56,
 74.41,
 67.5,
 61.93,
 59.55,
 60.47,
 64.55,
 55.58,
 59.7,
 74.45,
 54.35,
 74.04,
 55.73,
 72.01,
 60.04,
 70.8,
 29.96,
 55.16,
 54.31,
 60.82,
 65.5,
 67.02,
 44.92,
 65.51,
 67.88,
 64.57,
 57.18,
 63.36,
 66.21,
 55.96,
 67.63,
 64.51,
 63.58,
 61.06,
 70.79,
 68.37,
 61.78,
 48.96,
 65.49,
 62.84,
 38.19,
 71.9,
 64.73,
 49.49,
 7.37,
 3.39,
 60.12,
 49.42,
 40.78,
 45.87,
 49.7,
 50.9,
 62.71,
 61.55,
 33.59,
 56.26,
 64.01,
 37.89,
 70.4,
 67.98,
 57.52,
 62.16,
 59.98,
 65.94,
 24

In [6]:
# 原始字符串
s = "12.25 14.04 15.27 15.14 13.74 20.36 18.34 24.44 36.48 22.97 20.9 21.6 22.23 19.4 20.84 23.57 20.52 23.7 19.71 19.42 23.68 29.02 28.33 20.75 23.1 21.02 21.23 19.49 21.59 23.39 25.09 27.69 25.62 29.1 25.34 25.06 25.18 27.64 20.6 25.26 25.17 27.62 25.62 23.28 24.87 26.82 24.63 23.47 23.81 22.48 29.1 29.94 26.98 25.58 25.02 18.83 19.12 28.24 27.52 23.09 24.95 28.04 26.74 27.71 30.26 27.58 27.66 20.81 26.02 25.04 28.48 31.67 30.15 28.72 24.16 26.2 26.25 25.27 28.07 27.49 26.51 29.33 30.94 31.01 25.93 25.56 27.12 25.38 38.09 26.75 31.86 29.36 29.65 24.23 27.59 32.09 26.67 29.59 29.2 31.87 24 33.75 32.3 30.48 28.8 26.1 29.35 28.99 28.21 27.69 28.84 31.73 24.84 26.48 24.92 39.5 24.93 25.76 26.64 28.13 31.64 28.71 35.86 34.68 30.19 29.93 29.99 32.96 28.7 29.59 34.71 40.39 34.86 30.73 32.63 30.46 29.87 28.29 28.29 34.21 31.47 28.81 30.56 42.77 33.7 32.91 42.91 39.31 44.05 37.86 30.68 33.7 32.22 48.25 47.32 35.55 36.92 39.58 37.52 34.58 38.58 45.13 43.55 53.96 64.07 63.73 63.41 64.64 63.36 70.88"

# 使用 split() 方法按空格分割字符串
l = s.split()

# 使用 map() 方法把列表中的每个元素转换成浮点数
l = list(map(float, l))

# 打印结果
print(l)

[12.25, 14.04, 15.27, 15.14, 13.74, 20.36, 18.34, 24.44, 36.48, 22.97, 20.9, 21.6, 22.23, 19.4, 20.84, 23.57, 20.52, 23.7, 19.71, 19.42, 23.68, 29.02, 28.33, 20.75, 23.1, 21.02, 21.23, 19.49, 21.59, 23.39, 25.09, 27.69, 25.62, 29.1, 25.34, 25.06, 25.18, 27.64, 20.6, 25.26, 25.17, 27.62, 25.62, 23.28, 24.87, 26.82, 24.63, 23.47, 23.81, 22.48, 29.1, 29.94, 26.98, 25.58, 25.02, 18.83, 19.12, 28.24, 27.52, 23.09, 24.95, 28.04, 26.74, 27.71, 30.26, 27.58, 27.66, 20.81, 26.02, 25.04, 28.48, 31.67, 30.15, 28.72, 24.16, 26.2, 26.25, 25.27, 28.07, 27.49, 26.51, 29.33, 30.94, 31.01, 25.93, 25.56, 27.12, 25.38, 38.09, 26.75, 31.86, 29.36, 29.65, 24.23, 27.59, 32.09, 26.67, 29.59, 29.2, 31.87, 24.0, 33.75, 32.3, 30.48, 28.8, 26.1, 29.35, 28.99, 28.21, 27.69, 28.84, 31.73, 24.84, 26.48, 24.92, 39.5, 24.93, 25.76, 26.64, 28.13, 31.64, 28.71, 35.86, 34.68, 30.19, 29.93, 29.99, 32.96, 28.7, 29.59, 34.71, 40.39, 34.86, 30.73, 32.63, 30.46, 29.87, 28.29, 28.29, 34.21, 31.47, 28.81, 30.56, 42.77, 33.7, 3